In [ ]:
sqlContext = SQLContext(sc)

## Load house prices

In [ ]:
text_RDD = sc.textFile("/data/houses.txt")

In [ ]:
def mapper_parse_lines(line):
    """Parse line into (neighborhoood, price) pair"""
    words = line.split()
    return (words[1], float(words[2]), int(words[0]))

In [ ]:
house_prices_RDD = text_RDD.map(mapper_parse_lines)

In [ ]:
house_prices_RDD.collect()

In [ ]:
house_prices_df = sqlContext.createDataFrame(house_prices_RDD, ["neighborhood", "price", "bedrooms"])

In [ ]:
house_prices_df.show()

In [ ]:
house_prices_df.printSchema()

## Load inflation

In [ ]:
inflation_text_RDD = sc.textFile("/data/inflation.txt")

In [ ]:
def mapper_parse__inflation_lines(line):
    """Parse line into (neighborhoood, inflation) pair"""
    words = line.split()
    return (words[0], float(words[1]))

In [ ]:
inflation_RDD = inflation_text_RDD.map(mapper_parse__inflation_lines)

In [ ]:
inflation_RDD.collect()

In [ ]:
inflation_df = sqlContext.createDataFrame(inflation_RDD, ["neighborhood", "inflation"])

In [ ]:
inflation_df.printSchema()

## join

In [ ]:
house_prices_df.join(inflation_df).collect()

In [ ]:
def mapper_multiply_price_inflation(row):
    return (row["neighborhood"], row["price"] * row["inflation"])

In [ ]:
house_prices_nextyear_join = house_prices_df.join(inflation_df, "neighborhood").map(mapper_multiply_price_inflation)

In [ ]:
house_prices_nextyear_df = \
    house_prices_nextyear_join.select(
        "neighborhood", 
        (house_prices_nextyear_join.price * (1+house_prices_nextyear_join.inflation/100.)).alias("price"),
        "bedrooms"
    )

In [ ]:
house_prices_nextyear_df.show()

## reduce

In [ ]:
house_prices_nextyear_df.groupBy("neighborhood").sum("price").show()

## Exercise

Find the **mean** price for next year by number of bedrooms